# Network training

For our project, we constructed a Vision Transformer (ViT) instance that we named `GlowViT` to assess how light exposure affects Neural Network's result at classification.

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTModel, ViTForImageClassification, AutoImageProcessor, TrainingArguments, Trainer
from datasets import load_dataset

In [ ]:
class GlowViT(nn.Module):
    def __init__(self, pretrained_name="google/vit-base-patch16-224-in21k", num_classes=10):
        super(GlowViT, self).__init__()
        # Load pretrained ViT backbone
        self.vit = ViTForImageClassification.from_pretrained(
            pretrained_name,
            num_labels=num_classes)    

    def forward(self, pixel_values):
        # Pass through the ViT backbone
        outputs = self.vit(pixel_values=pixel_values)
        return outputs.logits

In [ ]:
# Hyperparameters
img_size = 32
patch_size = 4
num_classes = 10
batch_size = 64
learning_rate = 1e-3
num_epochs = 10

# Dataset and Dataloader
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model, Loss, Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlowViT(img_size=img_size, patch_size=patch_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
# Evaluate the Model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")